# **Matrices de rotación simples en un espacio tridimensional**

In [23]:
import numpy as np
from sympy import*
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
init_printing()

### **HWK4**

## **Clase para realizar Rotaciones**

In [24]:
class Rotaciones:


    def rotation_x(theta):

        return np.array([
            [1, 0, 0],
            [0, np.cos(theta), -np.sin(theta)],
            [0, np.sin(theta), np.cos(theta)]
        ])


    def rotation_y(theta):

        return np.array([
            [np.cos(theta), 0, np.sin(theta)],
            [0, 1, 0],
            [-np.sin(theta), 0, np.cos(theta)]
        ])


    def rotation_z(theta):

        return np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]
        ])


    def rotation_2d(theta):

        return np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])

# **Aplica múltiples rotaciones secuencialmente a un punto**

In [25]:
    def rotacion_masiva(cls, vector_rotaciones, punto_inicial):

        punto_actual = punto_inicial.copy()

        for angulo, eje in vector_rotaciones:
            # Convertir ángulo a radianes
            theta = np.deg2rad(angulo)

            # Seleccionar matriz de rotación según el eje
            if eje.lower() == 'x':
                rotacion = cls.rotation_x(theta)
            elif eje.lower() == 'y':
                rotacion = cls.rotation_y(theta)
            elif eje.lower() == 'z':
                rotacion = cls.rotation_z(theta)
            else:
                raise ValueError(f"Eje de rotación no válido: {eje}")

            # Aplicar rotación
            punto_actual = rotacion @ punto_actual

        return punto_actual

# **Aplica múltiples desplazamientos secuencialmente a un punto**

In [26]:
    def desplazamiento_masivo(cls, vector_desplazamientos, punto_inicial):

        punto_actual = punto_inicial.astype(np.float64).copy()

        for desplazamiento in vector_desplazamientos:
            punto_actual += np.array(desplazamiento)

        return punto_actual

# **Combina rotaciones y desplazamientos secuencialmente**

In [27]:
    def transformaciones_masivas(cls, lista_operaciones, punto_inicial):

        punto_actual = punto_inicial.copy()

        for operacion in lista_operaciones:
            if 'rotacion' in operacion[2].lower():
                # Es una rotación
                punto_actual = cls.rotacion_masiva([(operacion[0], operacion[1])], punto_actual)
            elif 'desplazamiento' in operacion[2].lower():
                # Es un desplazamiento
                punto_actual = cls.desplazamiento_masivo([operacion[1]], punto_actual)
            else:
                raise ValueError(f"Operación no reconocida: {operacion[2]}")

        return punto_actual

# **Visualiza puntos en un espacio 3D**

In [28]:
    def visualizar_puntos(puntos, etiquetas, titulo='Transformaciones'):

        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')

        colores = ['b', 'r', 'g', 'c', 'm', 'y', 'k']

        for i, (punto, etiqueta) in enumerate(zip(puntos, etiquetas)):
            color = colores[i % len(colores)]
            ax.scatter(punto[0], punto[1], punto[2], color=color, label=etiqueta)
            ax.quiver(0, 0, 0, punto[0], punto[1], punto[2], color=color, alpha=0.5)

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(titulo)
        ax.legend()
        plt.show()

# **Experimentos**

In [29]:
def experimento_1():
    """
    Demostración de rotación masiva.
    """
    print("Experimento 1: Rotación Masiva")
    punto_inicial = np.array([1, 1, 0])
    rotaciones = [
        (45, 'X'),
        (30, 'Y'),
        (60, 'Z')
    ]

    punto_final = TransformacionesRoboticas.rotacion_masiva(rotaciones, punto_inicial)

    TransformacionesRoboticas.visualizar_puntos(
        [punto_inicial, punto_final],
        ['Punto Inicial', 'Punto Final de Rotaciones']
    )
    print("Punto Inicial:", punto_inicial)
    print("Punto Final:", punto_final)

def experimento_2():
    """
    Demostración de desplazamiento masivo.
    """
    print("Experimento 2: Desplazamiento Masivo")
    punto_inicial = np.array([1, 1, 0])
    desplazamientos = [
        np.array([0.5, -0.5, 1]),
        np.array([-0.3, 0.2, 0.5])
    ]

    punto_final = TransformacionesRoboticas.desplazamiento_masivo(desplazamientos, punto_inicial)

    TransformacionesRoboticas.visualizar_puntos(
        [punto_inicial, punto_final],
        ['Punto Inicial', 'Punto Final de Desplazamientos']
    )
    print("Punto Inicial:", punto_inicial)
    print("Punto Final:", punto_final)

def experimento_3_conmutatividad():
    """
    Experimento para evaluar la conmutatividad de desplazamientos y rotaciones.
    """
    print("Experimento 3: Conmutatividad de Transformaciones")

    punto_inicial = np.array([1, 1, 0])

    # Desplazamientos
    d1 = np.array([0.5, -0.5, 1])
    d2 = np.array([-0.3, 0.2, 0.5])

    # Rotaciones
    r1 = (45, 'X')
    r2 = (30, 'Y')

    # Orden 1: d1 -> d2
    punto_1 = punto_inicial + d1
    punto_1 = punto_1 + d2

    # Orden 2: d2 -> d1
    punto_2 = punto_inicial + d2
    punto_2 = punto_2 + d1

    # Orden 1: r1 -> r2
    punto_3 = TransformacionesRoboticas.rotacion_masiva([r1], punto_inicial)
    punto_3 = TransformacionesRoboticas.rotacion_masiva([r2], punto_3)

    # Orden 2: r2 -> r1
    punto_4 = TransformacionesRoboticas.rotacion_masiva([r2], punto_inicial)
    punto_4 = TransformacionesRoboticas.rotacion_masiva([r1], punto_4)

    print("\nConmutatividad de Desplazamientos:")
    print("Orden d1 -> d2:", punto_1)
    print("Orden d2 -> d1:", punto_2)
    print("¿Son iguales los desplazamientos?", np.allclose(punto_1, punto_2))

    print("\nConmutatividad de Rotaciones:")
    print("Orden r1 -> r2:", punto_3)
    print("Orden r2 -> r1:", punto_4)
    print("¿Son iguales las rotaciones?", np.allclose(punto_3, punto_4))

# **Ejecutar experimentos**

In [ ]:
if __name__ == "__main__":
    print("Iniciando experimentos de Transformaciones \n")
    experimento_1()
    print("\n" + "-"*50 + "\n")
    experimento_2()
    print("\n" + "-"*50 + "\n")
    experimento_3_conmutatividad()

Conclusiones del Experimento de Conmutatividad:

Desplazamientos: Los desplazamientos son conmutativos (d1 -> d2 = d2 -> d1)

Rotaciones: Las rotaciones NO son conmutativas (r1 -> r2 ≠ r2 -> r1)

Esto significa que:

Cambiar el orden de desplazamientos no afecta el resultado final

Cambiar el orden de rotaciones SÍ afecta significativamente el resultado final